## <center>  CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 </center>

# <center> Optimizing Multi-core CPU Scheduling </center>

#### <center> Haoran Qiu (hqiu35@wisc.edu), Jian Wu (jwu384@wisc.edu), Tao Ji (tji24@wisc.edu), Yanting Liang (yliang73@wisc.edu) </center>
#### <center> Instructor: Prof. Laurent Lessard </center>
#### <center> May 7<sup>th</sup>, 2018 </center>

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
  1. [Problem Description](#1.1.-Problem-Description)
  2. [Goals and Objectives](#1.2.-Goals-and-Objectives)
  3. [Report Outline](#1.3.-Report-Outline)
2. [Mathematical Model](#2.-Mathematical-Model)
  1. [Concepts & Terminologies](#2.1.-Concepts-&-Terminologies)
  2. [Metric for Objectives](#2.2.-Metric-for-Objectives)
  3. [Perfect Information](#2.3.-Perfect-Information)
    1. [Knowing Arrival Time & Duration](#2.3.1.-Knowing-Arrival-Time-&-Duration)
    2. [Multicores](#2.3.2.-Multicores)
    3. [Process Dependency](#2.3.3.-Process-Dependency)
  4. [Robust Integer Programming](#2.4.-Robust-Programming)
    1. [Unknown Duration](#2.4.1.-Unknown-Duration)
    2. [Unknown Duration & Unknown Arrival Time](#2.4.2.-Unknown-Duration-&-Unknown-Arrival-Time)
  5. [Stochastic Programming](#2.5.-Stochastic-Programming)
    1. [Unknown Duration](#2.5.1.-Unknown-Duration)
    2. [Unknown Duration & Unknown Arrival Time](#2.5.2.-Unknown-Duration-&-Unknown-Arrival-Time)
3. [Solution](#3.-Solution)
4. [Results and Discussion](#4.-Results-And-Discussion)
5. [Conclusion](#5.-Conclusion)
6. [References](#6.-References)

*****

#### Note: ####
- **Interactive design**: In solution part, readers can choose whether they want to make their CPU scheduler to emphasize fairness, efficiency, or responsiveness by adjusting weight to corresponding metrics.
- **Latex rendering**: Please open our project in Jupyter notebook instead of viewing it in nbviewer.

## 1. Introduction ##

### 1.1. Problem Description ###

Sitting in front of your personal computer, perhaps you are playing a video game, browsing a website, and having a chat with your friends via WeChat at the same time. How could it be possible to open so many applications on a computer and run them at the same time? Who is in charge here? The answer is the operating system.

<img src="operating_system.jpg" width="450" height="400" />
###### <center>Fig. 1: Operating system makes it possible to run multiple applications at the same time</center> ######

The **Operating System**(OS) is a really important piece of software present in virtually all computer systems, which manages computer hardware and softwares and provides common resources for computer programs. A computer program is a piece of code but it lives as a process when it is being executed. One of the duties of an operating system is to schedule processes to run on the computer. In 1950s, process scheduling was not an issue because there is not so much process so that the OS can serve all users/processes one by one. When it came to 1960s - early 1980s, multiprogramming and time-sharing evolves([history](https://en.wikipedia.org/wiki/Timeline_of_operating_systems)). Process scheduling should handle multiple processes, swap job to avoid idleness. There will be many processes that want to be executed. Time-sharing OS allows many users/processes to share the computing resources such as main memory, input/output(I/O) devices, files, etc.

The OS has many components which work together to help carry out resource management duties. For example there is a memory manager(manages the usage of the main memory), various device drivers(carry out low-level interactions with hardware devices), I/O manager(handles data transfer between memory and I/O devices), process manager(creates and destroys processes), file system manager(manages the file system), etc. Another important component is the process scheduler, also known as the **Central Processing Unit**(CPU), which is the focus of this project.

<!--![A shot of processes running on a multicore CPU and the resource usage](cpu_scheduling.png =100x20)-->
<img src="cpu_scheduling.png" width="540" height="270" />
###### <center>Fig. 2: A shot of processes running on a 4-core CPU and the resource usage</center> ######

The job of the scheduler is to assign tasks to CPUs, i.e. what is the next process to be run on which CPU. For a computer, there could be multiple CPUs and each CPU could have multiple cores. For a process, it could generate a number of threads to be executed at the same time. Since every core in a CPU could run its assigned task at the same time, we could seen a core as the most basic execution unit. In addition, since a thread is similar as a process(and in some OS they are treat nearly the same way) we could seen a thread as a process. Therefore, the number of processes that can be run at the same time($X$) can be expressed by the multiplication of the number of CPUs($C$) and the number of cores per CPU($c_{i}$):

$$
X = \sum_{i=1}^{C} c_i
$$

However, things are really complicated in reality. There can be mainly two kinds of processes, one is called CPU-bound process and the other is called I/O-bound process, which should be treated differently. There may also be dependencies or conflicts among different processes. A process cannot continue running without another process returning a result, or two processes are using the same resource so that they cannot be scheduled to run at the same time. In addition, scheduling among multicores brings more cost caused by **context switch**. When the ongoing process is being replaced by another process before it is done, its state information should be stored to somewhere else(main memory or hard disk) and the new process' state information(if exists) should be moved to CPU. This process is called context switch. Switching between different cores leads to more cost. Last but not least, a CPU cannot predict when a process will finish its execution and it is not possible for a CPU to know when a particular process will arrive.

Considering all the complexities, we will present a general model in this report to solve this problem and offer an efficient solution for scheduling processes in multicore CPU.

### 1.2. Goals and Objectives ###

How the CPU, the bottleneck device in the whole computer, is scheduled, has a huge impact on the performance of the computer system. To make better decisions, the following objectives have to be met:

- Minimize the total turnaround time: turnaround time is the period of time between the arrival of a process and the time it is finished, which should be as small as possible;
- Maximize the number of processes that can be completed in a certain period of time, i.e. maximize the throughput.
- Be fair to every process: there should not be any process being wait for a long time without being executed;
- Be as responsive as possible: a process should be treated soon after its arrival;

### 1.3. Report Outline ###

First of all, several key terminologies will be introduced in section 2.1., followed by a section introducing the metric we used in our models to measure the objectives. This report will divide the whole problem into multiple stages, progressively solving more complex versions of this problem. Starting with something simple that we know the arrival time and the duraiton of each process(model 2.3.1.), then comes multicores(model 2.3.2.), then we deal with process dependencies(model 2.3.3.). After talking about models under perfect information, we will use two different models: robust programming(model 2.4.) and stochastic programming(model 2.5.) to deal with uncertainty in arrival time and duraiton of processes.

After presenting the models, we will give our solutions to each of the models and discuss about the result of the solutions. Different test cases and data can be come up with or drawn from the internet. Comparisons between our scheduling optimization and extant scheduling algorithms will also be made in the discussion section.

[shot]: https://github.com/tony-wj/Julia_hw/blob/master/cpu_scheduling.png

## 2. Mathematical Model ##

### 2.1. Concepts & Terminologies ###

Before we talking about the mathematical models, here are several terms and concepts the reader need to know:

- **CPU-bound(compute-bound) process**: when running, tends to use all the processor time that allocated to it;
- **I/O-bound process**: tends to use the processor only briefly before generating I/O request and relinquishes the processor;
    - Usually interactive, i.e. it is real-time and need **immediate reaction** from the OS, e.g. pressing a key on the keyboard
- **Process vs Thread**: they are both independent sequences of execution and the typical difference is that threads(of the same process) run in a shared memory space, while processes run in separate memory spaces;
- **Turnaround time**: the amount of time to execute a particular process(from submission to complete servicing);
    - $T_{turnaround}=T_{completion}-T_{arrival}$
- **Waiting time**: the amount of time a process was waiting in the ready queue(not being served);
- **Response time**: the amount of time it takes from when a process(job) is submitted to the first time it is scheduled(arrived but not getting served yet);
    - An important performance metric of interactive tasks
- **Throughput**: average number of processes that complete their execution per time unit;
- **Fairness**: all similar processes are treated the same, and even if processes are in different priority classes, no process should suffer indefinite postponement(starvation) due to scheduling;
- **Context-switch cost**: a context switch is the process of storing the state of a process or of a thread, so that it can be restored and execution resumed from the same point later(context-switch from one core to another is more expensive than swiching on the same core);
    - A kind of extra cost that we want to avoid

Although the realistic problem is actually really complicated and thorny, our key insights could help a lot the formulation of the model from the problem.

- All process could be seen as only using the CPU, i.e. no I/O. The reason behind is that jobs with I/O interact with the CPU for a tiny time period and then access the I/O so that they can be treated with each CPU burst(separated by I/O) as an independent job.
- Treat each thread as a process. Because they are both indenpendent execution of programs, both have context switch cost (and in fact in many modern OSs like Linux they are treated the same way).

### 2.2. Metric for Objectives ###

- Turnaround time: Let $A_{i}$, $F_{i}$, and $s_{i}$ be the arrival time, finish time, and duration of the i-th process respectively. Thus the metric for the turnaround time is:
<br><br>
$$Tu(\boldsymbol{X})=\frac{\sum_{i=1}^n (F_{i}-A_{i})}{\sum_{i=1}^n s_{i}}$$
<br>
- Fairness: Let $W_{i}$ be the total amount of waiting time for the i-th process(after the arrival and before complete), then fairness can be measured by the variance:
<br><br>
$$
F(\boldsymbol{X})=\sigma^2\\
\sigma^2 = \frac{\sum_{i=1}^{n}(W_i - \mu)^2} {n}\\
\mu = \frac{\sum_{i=1}^n W_{i}}{n}
$$
<br>
- Responsiveness: Let $E_{i}$ be the time when the process i first being executed, and the metric is:
<br><br>
$$R(\boldsymbol{X})=\sum_{i=1}^n E_i$$
<br>
- Cost: Let $\Psi(p, q, i, j)$ be the context switch cost of switching on core $p$ from process i to process j, and process j' previous state information is stored near core $q$. The cost of each context switch is:
<br><br>
$$
\Psi(p, q, i, j) = \left\{
        \begin{array}{ll}
            0 & i=j \\
            \psi_1 & i\neq j, p = q \\
            \psi_2 & i\neq j, p\neq q 
        \end{array}
    \right.
$$
<br>
The total context switch cost of single-core CPU is:
<br><br>
$$
C_1(\boldsymbol{X})=\sum_{t=2}^m \sum_{i -> j}^{n*n} \Psi(c, c, i, j)
$$
<br>
The total context switch cost of multi-core CPU is(given core set $Co$):
<br><br>
$$
C_2(\boldsymbol{X})=\sum_{t=2}^m \sum_{p, q\in Co} \sum_{i -> j}^{n*n} \Psi(p, q, i, j)
$$
<br>
- Throughput: Let $Th(x)$ be the throughput of in the time interval, and $\Phi_i$ denote the binary variable whether the process is completed. 
$$
Th(\boldsymbol{X})= \sum_{i=1}^n \Phi_i
$$

### 2.3. Perfect Information ###
#### 2.3.1. Knowing Arrival Time & Duration ####
We divide the time horizon into a number of time units. We only consider time in $[0, m]$ where $m$ is the maximum possible processing time. Let $\mathcal{T}$ denote the set of time units, and $|\mathcal{T}|=m$. Let $\mathcal{N}$ denote the set of tasks, where $|\mathcal{N}|=n$. Each process $n \in \mathcal{N}$ has following attributes: duration $s_n$, arrival time $t_n$. 

<img src="assignment_graph.png" width="400" height="400" />
<center>Fig. 3: An example of time units assignment</center>

The task scheduler assigns each process to the time units, hence we can formulate an Generalized Assignment Problem(GAP). Let binary decision variable $x_{ij}$ denote the j-th time unit is assigned to i-th process. The i-th process requires $s_i$ amount of time to process, and each time unit is at most associated with one process. We have:

$$
\begin{aligned}
\sum_{j=1}^m x_{ij} = s_i \qquad \forall i \in \mathcal{N}\\
\sum_{i=1}^n x_{ij} \leq 1 \qquad \forall j \in \mathcal{T}
\end{aligned}
$$

Note we can only assign time units after arrival of each process, hence the first constraint should be modified as:

$$
\begin{aligned}
\sum_{j = t_i}^m x_{ij} = s_i \qquad \forall i \in \mathcal{N}\\
\sum_{j = 1}^{t_i} x_{ij} = 0 \qquad \forall i \in \mathcal{N}
\end{aligned}
$$

Each feasible assignment $\mathbf{x}$ is measured by turnaround time $T(\mathbf{x})$, fairness $F(\mathbf{x})$, responsiveness $R(\mathbf{x})$. And cost $C_1(\mathbf{x})$ is included to penalize switch of processed between time units. 

The standard form of the model can be written as: 

$$
\begin{aligned}
\underset{x \in \mathbb{R^{n×m}}}{\text{maximize}}\qquad& Tu(x)+F(x)+R(x)+Th(x)-C_1(x)\\
\text{subject to:}\qquad& \sum_{j = t_i}^m x_{ij} - s_i \leq M \Phi_i - \epsilon(1-\Phi_i) \qquad \forall i \in \mathcal{N}\\
& \sum_{j = t_i}^m x_{ij} - s_i \geq m (1-\Phi_i)+ \epsilon \Phi_i \qquad \forall i \in \mathcal{N}\\
& \sum_{j = 1}^{t_i} x_{ij} = 0 \qquad \forall i \in \mathcal{N}\\
& \sum_{i=1}^n x_{ij} \leq 1 \qquad \forall j \in \mathcal{T}\\
& x_{ij} \in \{0,1\} \qquad \forall i \in \mathcal{N} \quad j \in \mathcal{T}
\end{aligned}
$$


#### 2.3.2. Multicores ####

Let $\mathcal{C}$ denote the set of cores, and $|\mathcal{C}|=c$. Binary decision variable $x_{ijk}$ donote the i-th process is assigned to the j-th time unit of core k. Another cost $C_2(x)$ is imposed if tasks is switched between cores. Now the standard form of the model can be rewritten as: 

$$
\begin{aligned}
\underset{x \in \mathbb{R^{n×m×c}}}{\text{maximize}}\qquad& T(x)+F(x)+R(x)+Th(x)-C_1(x)-C_2(x) \\
\text{subject to:}\qquad& \sum_{k = 1}^c \sum_{j = t_i}^m x_{ijk} - s_i \leq M \Phi_i - \epsilon(1-\Phi_i) \qquad \forall i \in \mathcal{N}\\
& \sum_{k = 1}^c \sum_{j = t_i}^m x_{ijk} - s_i \geq m (1-\Phi_i)+ \epsilon \Phi_i \qquad \forall i \in \mathcal{N}\\
& \sum_{i=1}^n x_{ijk} \leq 1 \quad \forall j \in \mathcal{T} \quad k \in \mathcal{C}\\
& x_{ijk} \in \{0,1\} \quad \forall i \in \mathcal{N} \quad j \in \mathcal{T} \quad k \in \mathcal{C}
\end{aligned}
$$

#### 2.3.3. Process Dependency ####
If process h is dependent on process g, which means process h should be hanlded strictly after g temporally. 

$$
\begin{align}
 j \cdot x_{hjk} \geq j \cdot x_{gjk}+1 \quad \forall j \in \mathcal{T} \quad k \in \mathcal{C}
\end{align}
$$

### 2.4. Robust Programming ###

In reality, the arrival time of the processes $\bf{t}$ and the time required to complete the processes $\bf{s}$ are random. Hence the scheduler does not have perfect information as discussed in above section. Regarding to the dynamic nature of the problem, we can further propose a robust programming model where these variables are subject to box constraints. This section is organized as following: First we propose a variant of the static model if process duration is unknown, and then we extend the model on the condition of unknow durations and unknown arrivals. 

#### 2.4.1. Unknown Duration ####
If the duration of the process is uncertain and is subject to some distribution. And assume somehow we know the distribution and the duration is subject to a box constraint. Let $\bar{s}$ denote some nominal value of the duration, and $u$ denote the uncertainty. 

$$
\begin{align}
s = \bar{s}+\rho^s \cdot u\\
0 \leq ||u||_{\infty} \leq 1
\end{align}
$$

#### 2.4.2. Unknown Duration & Unknown Arrival Time ####

If both the duration and arrival time of the process are uncertain. Similarly, we assume we know the variables are subject to a box constraint. Let $\bar{t}$ denote the nominal value of arrival time. 

$$
\begin{align}
t = \bar{t}+\rho^t \cdot u\\
0 \leq ||u||_{\infty} \leq 1
\end{align}
$$

### 2.5. Stochastic Programming ###

Use the Certainty Equivalence approach, use the expected value of random variables, i.e. $\bar{s}, \bar{t}$. 

#### 2.5.1. Unknown Duration ####

$$s_i = \bar{s_i}$$

#### 2.5.2. Unknown Duration & Unknown Arrival Time ####

$$s_i = \bar{s_i}$$
$$t_i = \bar{t_i}$$

## 3. Solution ##

*Modified by Tao Ji <jitaoivan@outlook.com> on May 5, 2018*

In [1]:
# Define input data

num_tasks = 5
A = [0, 2, 4, 6, 8]
D = [3, 6, 4, 5, 2]
max_time = minimum(A) + sum(D)                 # TO BE SETTLED

using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

@variable(m, x[1:num_tasks, 1:max_time], Bin)  # x[i, j] == 1 => task i gets executed at time j

@variable(m, 0 .<= F[1:num_tasks] .<= max_time, Int) # Finish time of each task

@variable(m, 0 .<= S[1:num_tasks] .<= max_time, Int) # Start time of each task
@variable(m, S_y[1:num_tasks, 1:max_time] >= 0, Int) # Assisting variable of start time
@variable(m, S_z[1:num_tasks, 1:max_time], Bin)      # Assisting variable of start time

@variable(m, 0 .<= W[1:num_tasks] .<= max_time, Int) # Waiting time of each task

for i = 1:num_tasks
    @constraint (m, W[i] == max_time - sum(x[i]))
end

# Expressing S
for i = 1:num_tasks
    for j = 1:max_time
        @constraint(m, S_y[i, j] == sum(x[i,1:j]))
    end
end
@constraint(m, S_y[i] - 1 .<= max_time*S_z[i] - 0.000001*(1-S_z[i])) # aTx − b ≤ Mz − ε(1−z)
@constraint(m, S_y[i] .>= 0.000001*S_z[i])                           # aTx −b ≥ m(1−z) + εz


for i = 1:num_tasks
    for j = 1:(A[i]-1)
        @constraint(m, x[i,j] == 0)            # A task should not be worked on before it arrives
    end
    for j = A[i]:max_time
        @constraint(m, F[i] >= j*x[i, j])      # Finish time is the last 1 of each task
    end
    @constraint(m, sum(x[i]) == 0)             # Total time quantums occupied by the task should be equal to its length
end

for j = 1:max_time
    @constraint(m, sum(x[:,j]) <= 1)           # Each time quantum should be occupied by at most 1 task
end

@objective()

INFO: Recompiling stale cache file C:\Users\tao\.julia\lib\v0.6\Gurobi.ji for module Gurobi.


Remember to make sure your code compiles! I will be running your code!

## 4. Results And Discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

## 6. Reference ##

1. Operating Systems, Wikipedia, refer to https://en.wikipedia.org/wiki/Operating_system
2. Time-sharing, Wikipedia, refer to https://en.wikipedia.org/wiki/Time-sharing
3. History of CPU process scheduling, refer to http://dave-reed.com/csc539.S05/Lectures/CPUsched.ppt
4. Context Switch, Wikipedia, refer to https://en.wikipedia.org/wiki/Context_switch#Cost